In [1]:
import numpy as np
import pandas as pd
%pylab inline
from flask import Flask, json, jsonify

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_json('cook_dataset.json')
data.head()

,cuisine,ingredients,name,rating
0,francuzskaya,"[Яблоко, Финики, Мед, Лимонный сок, Ваниль, Со...",Коблер,343
1,kitayskaya,"[Филе куриной грудки, Апельсины, Соевый соус, ...",Куриные кусочки в апельсиновом соусе с чили,13
2,kitayskaya,"[Белое сухое вино, Соевый соус, Чеснок, Жидкий...",Курица в кисло-сладком маринаде,66
3,armyanskaya,"[Вода, Пшеничная мука, Соль, Сливочное масло, ...",Армянский лаваш,569
4,kitayskaya,"[Стебель сельдерея, Зеленый лук перья, Соевый ...",Салат из сельдерея по‑китайски,62


In [3]:
class TrieNode(object):

    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.word_finished = False


def add(root, word: str):
    word = word.lower().replace('ё', 'е')
    node = root
    for char in word:
        found_in_child = False
        for child in node.children:
            if child.char == char:
                node = child
                found_in_child = True
                break
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            node = new_node
    node.word_finished = True


def prefix_descent(root, prefix: str):
    node = root
    # if prefix == '':
    # return False
    for char in prefix:
        char_not_found = True
        for child in node.children:
            if child.char == char:
                char_not_found = False
                node = child
                break
        if char_not_found:
            return False
    return node


def display(node, string, level, list_of_strings):
    if node.word_finished:
        res_string = ''.join(string)
        list_of_strings.append(res_string)
    else:
        for child in node.children:
            string.insert(level, child.char)
            display(child, string.copy(), level + 1, list_of_strings)
            string.pop()


def make_suggestions(root, prefix):
    prefix = prefix.lower()
    node = prefix_descent(root, prefix)
    list_of_strings = []
    string = []
    if type(node) == bool:
        list_of_strings.append(prefix)
    else:
        display(node, string, 0, list_of_strings)
        for i in range(len(list_of_strings)):
            list_of_strings[i] = prefix + list_of_strings[i]
    return list_of_strings



In [4]:
root = TrieNode('*')

In [5]:
for i in data['name']:
    add(root, i)

In [6]:
make_suggestions(root, 'Арм')

['армянский лаваш',
 'армянский соус для шашлыка',
 'армянский суп (кололик)',
 'армянский суп «спас»',
 'армянский салат из красной фасоли',
 'армянский шашлык из баранины',
 'армянский плов',
 'армянская пицца (ламаджо)']

In [9]:
make_suggestions(root, 'Па')

['панна-котта с клубникой',
 'панна-котта с миндальным соусом',
 'панна-котта с малиновым соусом',
 'панна-котта с пахтой, клубникой и черным перцем',
 'панна-котта адриана кетгласа',
 'панир',
 'панини с ветчиной и сыром',
 'пан кон томате',
 'пантумака',
 'панцанелла',
 'пан-банья (слоеный бутерброд в хлебе)',
 'панфорте (пирог из сухофруктов и орехов)',
 'паста из копченого баклажана',
 'паста из тофу для бутербродов',
 'паста из печеного чеснока',
 'паста из свежих помидоров',
 'паста из фасоли и перца (beans pepper cream)',
 'паста с чернилами каракатицы с морепродуктами в…',
 'паста с чесноком и оливковым маслом',
 'паста с чесноком, гребешками и рукколой',
 'паста с грибами',
 'паста с жареной тыквой баттернат',
 'паста с морепродуктами в сливочном соусе',
 'паста с морепродуктами и красным перцем',
 'паста с морепродуктами и пореем',
 'паста с морепродуктами по рецепту ресторана…',
 'паста с молодыми кабачками и помидорами черри',
 'паста с мидиями и креветками в медовом соусе'

In [10]:
make_suggestions(root, 'Салат')

['салат из сельдерея по‑китайски',
 'салат из сельдерея с грецкими орехами и…',
 'салат из селедки и апельсинов',
 'салат из спаржи с инжиром и грибами под соусом…',
 'салат из свеклы',
 'салат из свежего горошка',
 'салат из свежей моркови и свеклы',
 'салат из стручковой фасоли и зеленых орехов',
 'салат из стручковой фасоли с грушами и дор-блю',
 'салат из сладкого перца и мандарина',
 'салат из маринованной свинины с картофелем и…',
 'салат из маринованного осьминога',
 'салат из моркови и яблок с орехами',
 'салат из морепродуктов',
 'салат из морской капусты',
 'салат из молодой капусты, редиски и огурцов',
 'салат из мидий с орехами и яблоками',
 'салат из мидий с яблоками',
 'салат из мяса с луком и зернами граната',
 'салат из авокадо и крабового мяса под кедровым…',
 'салат из авокадо и огурца по‑индийски',
 'салат из авокадо с помидорами черри и виноградом',
 'салат из авокадо, томатов и базилика',
 'салат из апельсинов',
 'салат из апельсина и авокадо',
 'салат из ананасов 

In [ ]:
app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False


@app.route('/')
def hello():
    return 'start to type a name of the movie and get suggestions'


@app.route('/get_suggest/<prefix>', methods=['GET'])
def suggestions(prefix):
    suggest = make_suggestions(root, prefix)
    sug_string = ''
    for i in suggest:
        sug_string += i + '<br/>'
    return json.dumps(sug_string[:-1])


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Nov/2018 18:48:21] "GET //get_suggest/%D1%81%D1%83%D0%BF HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:48:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:48:29] "GET //get_suggest/dga HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:48:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:49:32] "GET /get_suggest/se HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 18:49:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:49:38] "GET /get_suggest/%D1%81%D1%83%D0%BF HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 18:49:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:49:57] "GET /get_suggest/%D0%BB%D0%B0%D0%B2%D0%B0%D1%88 HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 18:49:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Nov/2018 18:50:04] "GET /get_suggest/%D0%B1%D0%BE%D1%80%D1%89 HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2018 18:50:04] "GET /favicon.ico H